In [1]:
from app.dotenv import base_dir, data_dir,golden_data_dir
# from app.model.model import  Model
# import os
import pandas as pd
import numpy as np
from pandas import DataFrame
# set the max columns to none
pd.set_option('display.max_columns', None)
# set the max columns to none
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
# print('dotenv params:', {os.getenv('PARAM1')})
print('base_dir:', base_dir)
print('data_dir:', data_dir)
print('golden_data_dir:', golden_data_dir)

# model = Model()
# print('model:', model)

import json

base_dir: /home/yishai/Desktop/ivrit_ai_llm/app
data_dir: /home/yishai/Desktop/ivrit_ai_llm/data
golden_data_dir: /home/yishai/Desktop/ivrit_ai_llm/golden_data


avg_logprob - the average log probability of the tokens in the transcription
estimating the confidence of the transcription. Lower values (more negative) indicate lower confidence, while values closer to zero indicate higher confidence in the transcription.
Values generally range between -1.0 and 0, below -0.5 is suspected below -1 is very suspected


no_speech_prob - probability is high (close to 1), it suggests that the model believes the segment is likely silence or background noise, rather than actual speech. This can be useful for filtering out non-speech parts.
Values range from 0 to 1, where 0 indicates a high likelihood that the segment contains speech, and 1 indicates a high probability that the segment contains no speech.

compression_ratio - measure of how much the output text has been "compressed" compared to the input audio in terms of information content. Higher values might indicate potential issues, such as long stretches of repeated text or overly verbose output.
Typically ranges from 1.0 to 2.5. A value of 1.0 means that the transcription is roughly equal in length to the original audio input, while higher values indicate more compression (shorter transcribed text relative to the audio duration).

Suggested Workflow:

Filter Low Confidence Segments: Identify segments with a low avg_logprob or a high no_speech_prob and prioritize these for refinement by the LLM.

Fix Repetitive or Garbled Transcriptions: Use the compression_ratio to detect where the transcribed text might be problematic (too verbose, repeated words) and ask the LLM to clean up these sections.

Time-based Post-Processing: Use the start and end timestamps to make sure the LLM processes segments in the right sequence, especially if you are breaking up the transcription for batch processing.

#### load df from file

1. add wanted attribute columns & remove nulls  
2. drop columns
3. remove formatting chracters


In [2]:
df = pd.read_csv(f'{golden_data_dir}/crowd-transcribe-v4-enriched.csv')
ind = ~df['attrs'].isnull()
df.loc[ind,'attrs'] = df.loc[ind,'attrs'].apply(lambda x: json.loads(x))
df = df.loc[((~df['sentence'].isnull())*(~df['orig_sentence'].isnull()))]
# verify nulls are dropped
df['sentence'].loc[(~(~df['sentence'].isnull())*(~df['orig_sentence'].isnull()))]


Series([], Name: sentence, dtype: object)

In [3]:
vals = {'avg_logprob':-0.5,'no_speech_prob':0.5,'compression_ratio':2.0}
inds = {}
for key,value in vals.items():
    df[key] = value
    # df.loc[0,'attrs']['segments'][0]['avg_logprob']
    df[key] = None
    df.loc[ind,key] = df.loc[ind,'attrs'].apply( lambda x:  x['segments'][0][key] )
    # data = df.loc[ind,key].loc[df.loc[ind,key]>value]
    inds[key] = (df.loc[ind,key].loc[df.loc[ind,key]>value].index,df.loc[ind,key].loc[df.loc[ind,key]<value].index)    
    print(len(inds[key][0]))


111858
1901
5307


drop columns

In [4]:
df.drop(columns=['attrs','is_retranscribe','uuid'],inplace=True)
df.columns

Index(['orig_sentence', 'sentence', 'source', 'episode', 'avg_logprob',
       'no_speech_prob', 'compression_ratio'],
      dtype='object')

remove chracters function

In [5]:
def remove_chars(df: DataFrame,columns,remove_ords =[],remove_chars =[],replace_char = '') -> DataFrame:

    def replace_chars_from_col(col,in_chars,replace_char = '',is_ord = False):
        for ch in in_chars:
            if is_ord:
                col = col.str.replace(chr(ch),replace_char)
            else:
                col = col.str.replace(ch,replace_char)
            
        # for ch in ['  ','-']:
        #     col = col.str.replace(ch,' ')
        return col

    
    for col in columns:
        df[col] = df[col].str.lower()
        df[col] = replace_chars_from_col(df[col].copy(),remove_ords,replace_char = replace_char,is_ord = True)

        df[col] = replace_chars_from_col(df[col].copy(),remove_chars,replace_char = replace_char,is_ord = False)
    return df

remove error characters

In [6]:





def print_existing_unique_chars(cols):
    
    def cols_unique_characters(cols,print_ord = False):
        uniq_chars = sorted(set(''.join(''.join(col) for col in cols)))
        if print_ord:
            return [ord(ch) for ch in uniq_chars]
        return uniq_chars
    
    uniq1 = cols_unique_characters(cols)
    uniq2 = cols_unique_characters(cols,print_ord = True)
    print ('existing chars and their ords') 
    print('\t'.join([str(x)+',' for x in uniq1]))
    print('\t'.join([str(x)+',' for x in uniq2]))
    

# error characters:

error_ords =[]
error_ords1 = [1470,1468,1465,1463,1462,1460,1099,1464,47,1523,	1524,	1575,	1576,	1578,	1582,	1585,	1603,	1606,	8211,	8212,	8230,	9834,	9835,	20126,	21478,	24555,	26126,	26449,	27794,	29978,	35201,	38590,	40636,	46988,	47673,	49324,	49900,	50508,	50520,	50612,	50880,	51032,	51060,	65533,]
error_ords += error_ords1
# # latin letters - might be usefull to keep
# remove_ords2 = [380,	945,	950,	956,	959,	960,	1072,	1073,	1074,	1075,	1076,	1077,	1080,	1082,	1083,	1086,	1087,	1088,	1089,	1091,	1096,	1100]
# remove_ords += remove_ords2

error_chars = ['\u202b','\u200f','\u200e','\n']

   
df = remove_chars(df.copy(),columns = ['orig_sentence','sentence'],
                  remove_ords = error_ords,remove_chars = error_chars,
                  replace_char = '')

df = remove_chars(df.copy(),columns = ['orig_sentence','sentence'],
                  remove_chars = ['  '], replace_char = ' ')        
print_existing_unique_chars([df['orig_sentence'].copy(),df['orig_sentence'].copy()])

existing chars and their ords
 ,	!,	",	$,	%,	&,	',	(,	),	+,	,,	-,	.,	0,	1,	2,	3,	4,	5,	6,	7,	8,	9,	:,	=,	?,	[,	],	a,	b,	c,	d,	e,	f,	g,	h,	i,	j,	k,	l,	m,	n,	o,	p,	q,	r,	s,	t,	u,	v,	w,	x,	y,	z,	ż,	α,	ζ,	μ,	ο,	π,	а,	б,	в,	г,	д,	е,	и,	к,	л,	о,	п,	р,	с,	у,	ш,	ь,	א,	ב,	ג,	ד,	ה,	ו,	ז,	ח,	ט,	י,	ך,	כ,	ל,	ם,	מ,	ן,	נ,	ס,	ע,	ף,	פ,	ץ,	צ,	ק,	ר,	ש,	ת,
32,	33,	34,	36,	37,	38,	39,	40,	41,	43,	44,	45,	46,	48,	49,	50,	51,	52,	53,	54,	55,	56,	57,	58,	61,	63,	91,	93,	97,	98,	99,	100,	101,	102,	103,	104,	105,	106,	107,	108,	109,	110,	111,	112,	113,	114,	115,	116,	117,	118,	119,	120,	121,	122,	380,	945,	950,	956,	959,	960,	1072,	1073,	1074,	1075,	1076,	1077,	1080,	1082,	1083,	1086,	1087,	1088,	1089,	1091,	1096,	1100,	1488,	1489,	1490,	1491,	1492,	1493,	1494,	1495,	1496,	1497,	1498,	1499,	1500,	1501,	1502,	1503,	1504,	1505,	1506,	1507,	1508,	1509,	1510,	1511,	1512,	1513,	1514,


remove empty columns

In [7]:

for col in ['orig_sentence','sentence']:
    ind = df[col].str.len()==0
    df = df.loc[~ind]   



save temporary copy

In [8]:
df1 = df.copy()

#### baseline results - filter out samples without mistakes

WER - word error rate

calculate how well the hypothesis (orig_sentence) matches the correct reference (sentence)

WER = (S+D+I)/N

In [9]:
# from jiwer import wer

# reference = "the correct translation"
# hypothesis = "your corrected sentence"

# error_rate = wer(reference, hypothesis)
# print(f"Word Error Rate: {error_rate}")
import jiwer


transforms = jiwer.Compose(
    [
        jiwer.ExpandCommonEnglishContractions(),
        jiwer.RemoveEmptyStrings(),
        jiwer.ToLowerCase(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
        jiwer.RemovePunctuation(),
        jiwer.ReduceToListOfListOfWords(),
    ]
)


df['wer'] = df.apply(lambda x: jiwer.wer(x['sentence'],x['orig_sentence'],
                                   truth_transform=transforms, hypothesis_transform=transforms,
                                   )
                     ,axis = 1)

In [10]:
LEN = len(df.loc[df['wer']==0.0])
print('zero wer:',LEN,' percentage:',LEN/len(df))
# zero wer: 78808  percentage: 0.50085479862978


df = df.loc[df['wer']>0.0]

zero wer: 78808  percentage: 0.50085479862978


#### pre processing - remove formatting:

they are not errors - just to find mistake senetences

later they should be added again

remove formatting characters

In [11]:


formatting_ords = [32,	33,	34,	36,	37,	38,	39,	40,	41,	43,	44,	45,	46,58,	61,	63,	91,	93,]
formatting_chars = 'ההה'
df = remove_chars(df.copy(),columns = ['orig_sentence','sentence'],
                  remove_ords = formatting_ords,
                #   remove_chars = formatting_chars,    
                  replace_char = ' ')    
print_existing_unique_chars([df['orig_sentence'].copy(),df['orig_sentence'].copy()])

existing chars and their ords
 ,	0,	1,	2,	3,	4,	5,	6,	7,	8,	9,	a,	b,	c,	d,	e,	f,	g,	h,	i,	j,	k,	l,	m,	n,	o,	p,	q,	r,	s,	t,	u,	v,	w,	x,	y,	z,	ż,	α,	ζ,	μ,	ο,	π,	а,	б,	в,	г,	д,	е,	и,	к,	л,	о,	п,	р,	с,	у,	ш,	ь,	א,	ב,	ג,	ד,	ה,	ו,	ז,	ח,	ט,	י,	ך,	כ,	ל,	ם,	מ,	ן,	נ,	ס,	ע,	ף,	פ,	ץ,	צ,	ק,	ר,	ש,	ת,
32,	48,	49,	50,	51,	52,	53,	54,	55,	56,	57,	97,	98,	99,	100,	101,	102,	103,	104,	105,	106,	107,	108,	109,	110,	111,	112,	113,	114,	115,	116,	117,	118,	119,	120,	121,	122,	380,	945,	950,	956,	959,	960,	1072,	1073,	1074,	1075,	1076,	1077,	1080,	1082,	1083,	1086,	1087,	1088,	1089,	1091,	1096,	1100,	1488,	1489,	1490,	1491,	1492,	1493,	1494,	1495,	1496,	1497,	1498,	1499,	1500,	1501,	1502,	1503,	1504,	1505,	1506,	1507,	1508,	1509,	1510,	1511,	1512,	1513,	1514,


remove starting charcters - not sure it matters

In [12]:
# # remove_chars =[' ']

# # data = ['שלום מה שלומך',' שלום מה שלומך','שלום מה שלומך ','שלום מה שלומך']
# # df1 = pd.DataFrame([s],columns = ['orig_sentence'])
# # print(df1.values)

# def remove_start_end_chars(df: DataFrame,columns,remove_chars =[],) -> DataFrame:
#     def remove_start_end_chars_from_col(col,in_chars):
#         for ch in in_chars:
#             ind = col.str.startswith(ch)
#             col.loc[ind] = col.loc[ind].str[1:]
#             ind = col.str.endswith(ch)
#             col.loc[ind] = col.loc[ind].str[:-1]
            
#             # col = col.str.replace('^'+ch,'')
#             # col = col.str.replace(ch+'$','')
#         return col
#     for col in columns:
#         df[col] = remove_start_end_chars_from_col(df[col].copy(),remove_chars)

#     return df
    

# df = remove_start_end_chars(df.copy(),['orig_sentence','sentence'],remove_chars = ['     ','    ','   ','  ',' '])





remove shouts and laughter

from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("yam-peleg/Hebrew-Mistral-7B-200K")
# model = AutoModelForCausalLM.from_pretrained("yam-peleg/Hebrew-Mistral-7B-200K")

text = "שלום!מהשלמהיום?"
# input_ids = tokenizer(text, return_tensors="pt")#.to("cuda")
# print(input_ids)
tokens = tokenizer(text )
print(tokens['input_ids'])  
decoded_text = tokenizer.decode(tokens['input_ids'])
print(decoded_text)  

encoded_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'])

# Print the tokens
print(encoded_text)

#### analyisis

1. seperate data into seperate words (remove formatting)
2. find the hebrew words errors
3. find the errors location in original df

seperate data into seperate words

In [13]:

def remove_numbers(uniq):
    return [x for x in uniq if not x.isnumeric()]
def df_unique_words(col):
    joined = ' '.join(col)
    uniq =  sorted(list(set(joined.split(' '))))
    uniq = remove_numbers(uniq)
    
    
    return uniq
unique_words = df_unique_words(df['orig_sentence'].copy().tolist())
# uniq = df_unique_words(uniq)
# uniq = df_unique_words(uniq)
# print(uniq)
# uniq

In [14]:



def find_hebrew_words(uniq):
    hebrew_words = [word for word in uniq if any(ord(ch)>1487 for ch in word)]
    return hebrew_words

def find_error_words(hebrew_words):
    errors = []
    end_letters = set([1509,1507,1503,1501])
    errors += [word for word in hebrew_words if any(ord(ch) in end_letters for ch in word[:-1])]
    
    # english_letters = set(range(97,123))
    # errors += [word for word in hebrew_words if any(ord(ch) in english_letters for ch in word)]
    
    
    # todo: א vs ע ה
    # todo: 'כ' vs 'ק'
    # todo:
    # todo: compare to dictionairy
    # todo: words with foreign letters  - (might be translated fonetically)
    # todo: words with numbers
    # todo: words with special characters
    
    
    
    
    
    return errors
hebrew_words = find_hebrew_words(unique_words)
errors = find_error_words(hebrew_words)
# errors

In [15]:
def find_error_words_df_location(col,errors):
    # errors_set = set(errors)
    
    error_location = {}
    for error in errors:
        ind = col.str.contains(error)
        error_location[error] = df.loc[ind].index.tolist()
    # todo: get the clean formatted sentence
    # todo: get the location of the error in the sentence  
    return error_location

error_location = find_error_words_df_location(df['orig_sentence'].copy(),errors)
{k: len(v) for k, v in sorted(error_location.items(), key=lambda item: len(item[1]),
                              reverse=True)}

{'בלוםפילד': 3,
 'יוםך': 2,
 'כיףי': 2,
 'מץט': 2,
 'עםך': 2,
 'קוץק': 2,
 'בבלוםפילד': 1,
 'בוואץמוק': 1,
 'ביןך': 1,
 'במוןиру': 1,
 'דוםס': 1,
 'היוםיומיים': 1,
 'הקוץקר': 1,
 'ותבציץו': 1,
 'ותכיןי': 1,
 'יעדיףו': 1,
 'יקבץך': 1,
 'ירוץו': 1,
 'ישיםו': 1,
 'כיףים': 1,
 'כןהאמיתי': 1,
 'כןים': 1,
 'מיווןי': 1,
 'מןבור': 1,
 'סולםים': 1,
 'עליהםו': 1,
 'עםμοיח': 1,
 'עםה': 1,
 'עםו': 1,
 'ערומיםך': 1,
 'קץר': 1,
 'רובןפינסקי': 1,
 'רייליףמן': 1,
 'שאילביץר': 1,
 'שאיןבעלות': 1,
 'שוטרוץקין': 1,
 'תריםו': 1}

In [19]:
error_location

{'בבלוםפילד': [135193],
 'בוואץמוק': [69963],
 'ביןך': [75561],
 'בלוםפילד': [54165, 130521, 135193],
 'במוןиру': [99266],
 'דוםס': [116433],
 'הקוץקר': [68705],
 'ותבציץו': [37089],
 'ותכיןי': [21437],
 'יוםך': [64789, 66140],
 'יקבץך': [33295],
 'ירוץו': [8794],
 'ישיםו': [93568],
 'כיףי': [38441, 69633],
 'כיףים': [69633],
 'כןהאמיתי': [143138],
 'כןים': [42977],
 'מיווןי': [122720],
 'מןבור': [63168],
 'מץט': [92021, 115575],
 'סולםים': [27921],
 'עליהםו': [41291],
 'עםμοיח': [69783],
 'עםה': [68412],
 'עםו': [137060],
 'עםך': [98898, 131502],
 'ערומיםך': [36278],
 'קוץק': [68705, 110085],
 'קץר': [54643],
 'רובןפינסקי': [116508],
 'רייליףמן': [62694],
 'שאילביץר': [32572],
 'שאיןבעלות': [54331],
 'שוטרוץקין': [129034],
 'תריםו': [118908]}

In [23]:
df.loc[42977].values

array(['לגמרי כןים לגבי הכל', 'לגמרי כנים לגבי הכל', 'Geekonomy',
       '2022.04.12 פרק #539 - נטע בונדי נהנית לקודד', -0.3959960862994194,
       0.00720977783203125, 0.8780487804878049, np.float64(0.25)],
      dtype=object)

#### dictabert MLM

https://arxiv.org/abs/2308.16687

In [ ]:
# model_add = "dicta-il/dictabert"
model_add = f"{data_dir}\\dictabert"
print(model_add)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM
from torch import topk, tensor
def process_sentence(sentence: str, current_word, new_word):
    return sentence.replace(current_word, new_word), sentence.split(' ').index(current_word)+1


def get_top_k_words(model, tokenizer, top_k: int = 3,sentence: str = '',word_location: int = 0):
    output = model(tokenizer.encode(sentence, return_tensors='pt'))
    # the [MASK] is the 7th token (including [CLS])
    top_k = topk(output.logits[0, word_location, :], top_k)[1]
    return tokenizer.convert_ids_to_tokens(top_k)




tokenizer = AutoTokenizer.from_pretrained(model_add)
model = AutoModelForMaskedLM.from_pretrained(model_add)
model.eval()

# sentence = 'בשנת 1948 השלים אפרים קישון את התמחותו בפיסול מתכת ובתולדות האמנות והחל לפרסם מאמרים הומוריסטיים'


sentence = remove_special_chars(df['orig_sentence'].loc[135193])
print(sentence)
print('_'*150)
current_word = 'בבלוםפילד'
bert_input, word_location = process_sentence(sentence, current_word , new_word = '[MASK]')

tokens = get_top_k_words(model, tokenizer, top_k = 3,sentence = bert_input,word_location = word_location)
for token in tokens:
    bert_output,_ = process_sentence(bert_input, current_word = '[MASK]', new_word = token)
    print(bert_output)



'בשנת 1948 השלים אפרים קישון את 
התמחותו 
בפיסול מתכת ובתולדות האמנות והחל לפרסם מאמרים הומוריסטיים'

#### other

KeyboardInterrupt: 

BERTScore is excellent for semantic similarity and will show you how much your corrections make the meaning closer to the reference.
BLEU and ROUGE are good if you care about exact word matches (BLEU) or content recall (ROUGE).
WER gives a direct error count, showing how many errors remain.
Cosine Similarity between sentence embeddings provides a more holistic view of how semantically close your corrections are to the reference.
If your corrections focus on improving the meaning and not just the wording, BERTScore or Cosine Similarity would likely be the best metrics. If you're aiming for exact match or word-level accuracy, go with BLEU, ROUGE, or WER.

bleu

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Reference translations (can have multiple references)
references = [
    ['the', 'cat', 'is', 'on', 'the', 'mat'],
    ['there', 'is', 'a', 'cat', 'on', 'the', 'mat']
]

# Candidate translation (generated by a machine)
candidate = ['the', 'cat', 'is', 'on', 'the', 'mat']

# Calculate BLEU score
bleu_score = sentence_bleu(references, candidate, smoothing_function=SmoothingFunction().method1)

print(f"BLEU Score: {bleu_score:.4f}")


In [ ]:
from transformers import AutoTokenizer, AutoModel
from torch.nn.functional import cosine_similarity

# Load model and tokenizer
model = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Sentences to compare
ref_sentence = "the correct translation"
corrected_sentence = "your corrected sentence"

# Tokenize and get embeddings
ref_tokens = tokenizer(ref_sentence, return_tensors="pt")
corrected_tokens = tokenizer(corrected_sentence, return_tensors="pt")

with torch.no_grad():
    ref_embedding = model(**ref_tokens).last_hidden_state.mean(dim=1)
    corrected_embedding = model(**corrected_tokens).last_hidden_state.mean(dim=1)

similarity = cosine_similarity(ref_embedding, corrected_embedding).item()
print(f"Cosine Similarity: {similarity}")


bert score

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(f"{data_dir}\\dictabert")
model = AutoModelForMaskedLM.from_pretrained(f"{data_dir}\\dictabert")

model.eval()

sentence = 'בשנת 1948 השלים אפרים קישון את [MASK] בפיסול מתכת ובתולדות האמנות והחל לפרסם מאמרים הומוריסטיים'

output = model(tokenizer.encode(sentence, return_tensors='pt'))
# the [MASK] is the 7th token (including [CLS])
import torch
top_2 = torch.topk(output.logits[0, 7, :], 2)[1]
print('\n'.join(tokenizer.convert_ids_to_tokens(top_2))) # should print מחקרו / התמחותו 

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM, BertModel
from bert_score import BERTScorer,score

# Example texts
reference = "This is a reference text example."
candidate = "This is a candidate text example."
# BERTScore calculation
# scorer = BERTScorer(model_type='')
# P, R, F1 = scorer.score([candidate], [reference])





# # Load the Hebrew BERT model and tokenizer
model_name = f"{data_dir}\\bert-base-uncased"
# Define the reference and candidate sentences in Hebrew
reference_sentences = ['הטקסט המדויק הזה הוא המושלם.']
candidate_sentences = []'המשפט הזה הוא הטוב ביותר.']
lang = 'he'


# model_name = f"{data_dir}\\bert-base-uncased"
# reference_sentence = "This is a reference text example."
# candidate_sentence = "This is a candidate text example."
lang = 'en'




# required when loaded from local instead of hugginface
model = AutoModel.from_pretrained(model_name)
num_layers = model.config.num_hidden_layers
P, R, F1 = score(candidate_sentence, reference_sentence,model_type=model_name,num_layers = num_layers,rescale_with_baseline=True,lang='he')
print(f"BERTScore Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity




# Function to get token embeddings for a list of sentences
def get_token_embeddings(sentences,model,tokenizer):
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        # Tokenize the input sentences
        inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
        # Get the embeddings
        outputs = model(**inputs)
        # Get the embeddings for all tokens
        embeddings = outputs.last_hidden_state
    return embeddings

# Compute BERTScore Precision, Recall, and F1
def compute_bert_score(reference, candidate,model_name):
    print('model_name:', model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    ref_embeddings = get_token_embeddings([reference],model,tokenizer)
    cand_embeddings = get_token_embeddings([candidate],model,tokenizer)
    
    # embeddings for a list of sentences (not by token)
    similarity =  cosine_similarity(ref_embeddings.mean(dim=1),cand_embeddings.mean(dim=1))
    BERTScore = similarity[0][0]
    
    # Compute cosine similarities between each token in the candidate and reference
    ref_tokens = tokenizer(reference, return_tensors='pt')['input_ids'].squeeze().tolist()
    cand_tokens = tokenizer(candidate, return_tensors='pt')['input_ids'].squeeze().tolist()
    
    ref_embeddings = ref_embeddings.squeeze(0)
    cand_embeddings = cand_embeddings.squeeze(0)
    
    
    # Token-level similarity matrix
    similarity_matrix = cosine_similarity(cand_embeddings.numpy(), ref_embeddings.numpy())
    
    # Compute precision
    precision_scores = []
    for i, cand_token in enumerate(cand_tokens):
        cand_emb = cand_embeddings[i].unsqueeze(0).numpy()
        similarities = similarity_matrix[i]
        max_similarity = max(similarities)
        precision_scores.append(max_similarity)
    
    precision = np.mean(precision_scores)
    
    # Compute recall
    recall_scores = []
    for j, ref_token in enumerate(ref_tokens):
        ref_emb = ref_embeddings[j].unsqueeze(0).numpy()
        similarities = similarity_matrix[:, j]
        max_similarity = max(similarities)
        recall_scores.append(max_similarity)
    
    recall = np.mean(recall_scores)
    
    # F1 score as the harmonic mean of precision and recall
    if precision + recall > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0.0
    
    return precision, recall, f1,BERTScore


# Load the Hebrew BERT model and tokenizer
model_name = f"{data_dir}\\bert-base-uncased"
# Define the reference and candidate sentences in Hebrew
reference_sentence = 'הטקסט המדויק הזה הוא המושלם.'
candidate_sentence = 'המשפט הזה הוא הטוב ביותר.'

# model_name = f"{data_dir}\\bert-base-uncased"
# reference_sentence = "This is a reference text example."
# candidate_sentence = "This is a candidate text example."


# Compute BERTScore
precision, recall, f1,BERTScore = compute_bert_score(reference_sentence, candidate_sentence,model_name)

# Print BERTScore Precision, Recall, and F1
print(f"BERTScore Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, BERTScore: {BERTScore:.4f}")


#### next steps

In [ ]:

df = df1.copy()
df.rename(columns={'orig_sentence':'x_orig', 'sentence':'y_orig'}, inplace=True)
df.drop(['uuid','is_retranscribe'], axis=1, inplace=True)
df = df.loc[:100]
df.head()



In [ ]:

df['x'] = remove_special_chars(df['x_orig'].copy())
df['y'] = remove_special_chars(df['y_orig'].copy())

# count words in df
df['x_count'] = df['x'].apply(lambda x: len(str(x).split()))
df['y_count'] = df['y'].apply(lambda x: len(str(x).split()))

df[['x','y']]

In [ ]:
def df_unique_words(df):
    return sorted(list(set(' '.join(df).split())))
print(df_unique_words(df['x']))
# df['x'].str.split('\s+', expand=True)
#                   .stack()
#                   .drop_duplicates()
#                   .tolist()
# )

In [ ]:
a = df['x'].loc[8].split(' ')
b = df['y'].loc[8].split(' ')
print(a)
print(b)

In [ ]:

arr = []
aa = []
bb=[]
i,j = len(a)-1,len(b)-1
while i>=0 and j>=0:    
    if a[i] == b[j]:
        aa.append(a[i])
        bb.append(b[j])
        i-=1
        j-=1
    elif a[i] in b:
        aa.append(a[i])
        i+=1
    else:
        aa.append(a[i])
        bb.append(None)
        bb.append(b[j])
        aa.append(None)
        
        i-=1
        j-=1
        

    
    #     
    #     bb.append(None)
    #     j+=1   
        
#     # else:
#     #     aa.append(a[i])
#     #     bb.append(None)
#     #     aa.append(a[i])
#     #     bb.append(None)
#     #     i+=1
#     #     j+=1

# for i,word in enumerate(a):
#     print(word)
#     print(i,np.where(b == word)[0])
#     print('_'*100)



In [ ]:
aa

In [ ]:
bb

In [78]:
def split_col(df,col,extra_cols=10):
    # Split the 'x' column into separate columns
    split_df = df[col].str.split(r'\s+', expand=True)
    # Rename the columns
    columns = [f'{i+1}{col}' if i+1>9 else f'0{i+1}{col}' for i  in range(split_df.shape[1]+extra_cols)]
    split_df.columns = columns[:split_df.shape[1]]
    
    # add extra empty columns for allowing to move columns later
    for col in  columns[split_df.shape[1]:]:
        split_df[col] = None  # You could use `None` instead of `pd.NA` if you prefer
    # Join the new columns back to the original DataFrame
    df = df.join(split_df)
    df.drop(col, axis=1, inplace=True)
    return df

df = split_col(df.copy(),'x')
df = split_col(df.copy(),'y')
df = df[sorted(df.columns)]

In [90]:
def move_row(row,from_col,name='x'):
    ind = row.index
    cols = [i for i in ind[row.notnull()] if i.endswith(name)][from_col-1:-1]
    last = int(cols[-1].replace(name,''))+1
    cols += [f'{last}{name}' if last>9 else f'0{last}{name}']
    row[cols[1:]] = row[cols[:-1]] 
    row[cols[0]] = None 
    
    
    # row = row.append(pd.Series([None]*(len(row),len(from_col)-len(row)), index=from_col)
    return row
    
# move_row(row=df.loc[0],from_col=1,to_col=1)

In [ ]:
print(df.loc[0])
print(move_row(row=df.loc[0].copy(),from_col=2))

In [ ]:
def decode_word(word):
    # Decode the Hebrew words - drop the first 1488 unicode value of aleph letter
    return [ord(char)-1488 for char in word]
def decode_pair(word1,word2):
    vector1 = decode_word(word1)  # Example vector for "example"
    vector2 = decode_word(word2)  # Example vector for "exemplary"
    LEN = max(len(vector1),len(vector2))
    vector1 += [0]*(LEN-len(vector1))
    vector2 += [0]*(LEN-len(vector2))
    print(vector1),print(vector2)
    correlation = pearsonr(vector1, vector2)[0]
    print(f"Pearson correlation coefficient: {correlation}")
    return vector1,vector2
vector1,vector2 = decode_pair('שלום','בא')

In [ ]:
from scipy.stats import pearsonr


# Example phonetic feature vectors
# In practice, these would be derived from a more complex phonetic analysis
# vector1 = decode_word('הקבלה')  # Example vector for "example"
# vector2 = decode_word('הקבלה')  # Example vector for "exemplary"

# Compute Pearson correlation coefficient



count number of words in each cell

In [ ]:
print (df['x'].str.replace(r'''[\-\!\@\#\$\%\^\&\*\(\)\_\+\[\]\;\'\.\,\/\{\}\:\"\<\>\?\|]''','')
                #   .replace(r'[\  \   \]',' ')
                  .str
                  .lower()
                  .str
                  .split('\s+', expand=True)
                  .stack()
                  .drop_duplicates()
                  .tolist()
)

In [51]:

# df['y_count'] = df['y'].apply(lambda x: len(str(x).split()))
remove [',','  '] in x
for  
df['x'] = df['x'].apply(lambda x: x.replace(',',''))


In [ ]:
df.head()